In [51]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
import time
import warnings
import random
from matplotlib import pyplot
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input, Dropout, Activation, Dense, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [52]:
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.ConvergenceWarning)

In [70]:
df = pd.read_csv("../../preprocessing/iot23_combined.csv", skiprows=lambda x: x > 0 and random.random() >= 0.25)
del df["Unnamed: 0"]
df = df.replace(r'^\s*$', np.nan, regex=True)
df


,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
2,2.999300,0,0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
3,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
4,2.998813,0,0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336252,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,DDoS,0,...,0,0,0,0,0,0,0,0,0,0
336253,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,DDoS,0,...,0,0,0,0,0,0,0,0,0,0
336254,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,DDoS,0,...,0,0,0,0,0,0,0,0,0,0
336255,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,DDoS,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
df['label'].unique()

array(['PartOfAHorizontalPortScan', 'Benign', 'C&C', 'Attack',
       'C&C-HeartBeat', 'Okiru', 'DDoS', 'C&C-Torii', 'C&C-FileDownload',
       'FileDownload', 'C&C-HeartBeat-FileDownload'], dtype=object)

In [72]:
conditions = [df.label == 'PartOfAHorizontalPortScan', df.label == 'Benign',df.label == 'Attack',df.label == 'C&C',df.label == 'Okiru',df.label == 'C&C-HeartBeat',df.label == 'DDoS',df.label == 'C&C-Torii',df.label == 'C&C-FileDownload',df.label == 'FileDownload',df.label ==  'C&C-HeartBeat-FileDownload',df.label ==  'C&C-Mirai']
# len(conditions)
choices = [1,2,3,4,5,6,7,8,9,10,11,12]

In [73]:
df['trueLabel'] = np.select(conditions,choices)

In [74]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = df['trueLabel']
# Y = df['label']
Y = pd.factorize(df['label'])[0].reshape(-1, 1)

In [75]:
scaler = MinMaxScaler()
normalized_x = scaler.fit_transform(X)
normalized_x

array([[1.46962071e-04, 2.37102742e-09, 3.58824205e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.46962071e-04, 2.37102742e-09, 3.58824205e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.87745411e-04, 2.37102742e-09, 3.58824205e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.46962071e-04, 2.37102742e-09, 3.58824205e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.46962071e-04, 2.37102742e-09, 3.58824205e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.46962071e-04, 2.37102742e-09, 3.58824205e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [76]:
# normalized_y = pd.factorize(df['label'])[0].reshape(-1, 1)

In [77]:
# normalized_y = scaler.fit_transform(Y)
# normalized_y

In [79]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, Y, random_state= 100, test_size=0.2)

In [80]:
model = Sequential()

In [81]:
model.add(Dense(2000, activation='relu',input_dim=24))
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(800,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(12, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [82]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 2000)              50000     
                                                                 
 dense_19 (Dense)            (None, 1500)              3001500   
                                                                 
 dropout_12 (Dropout)        (None, 1500)              0         
                                                                 
 dense_20 (Dense)            (None, 800)               1200800   
                                                                 
 dropout_13 (Dropout)        (None, 800)               0         
                                                                 
 dense_21 (Dense)            (None, 400)               320400    
                                                                 
 dropout_14 (Dropout)        (None, 400)              

In [83]:
start = time.time()
print('program start...')
print()

history = model.fit(X_train, Y_train, epochs = 10, batch_size=256, validation_data=(X_test,Y_test),verbose=1)

print()
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...

Epoch 1/10
1051/1051 [==============================] - 163s 154ms/step - loss: 0.8205 - accuracy: 0.7420 - val_loss: 0.7902 - val_accuracy: 0.7471
Epoch 2/10
1051/1051 [==============================] - 163s 155ms/step - loss: 0.8006 - accuracy: 0.7439 - val_loss: 0.7879 - val_accuracy: 0.7470
Epoch 3/10
1051/1051 [==============================] - 161s 153ms/step - loss: 0.7990 - accuracy: 0.7441 - val_loss: 0.7885 - val_accuracy: 0.7471
Epoch 4/10
1051/1051 [==============================] - 158s 150ms/step - loss: 0.7983 - accuracy: 0.7441 - val_loss: 0.7877 - val_accuracy: 0.7472
Epoch 5/10
1051/1051 [==============================] - 156s 148ms/step - loss: 0.7975 - accuracy: 0.7441 - val_loss: 0.7874 - val_accuracy: 0.7473
Epoch 6/10
1051/1051 [==============================] - 165s 157ms/step - loss: 0.7971 - accuracy: 0.7443 - val_loss: 0.7872 - val_accuracy: 0.7474
Epoch 7/10
 361/1051 [=========>....................] - ETA: 1:46 - loss: 0.7986 - accuracy: 0

KeyboardInterrupt: 